<a href="https://colab.research.google.com/github/fengivan/382L-HW5/blob/master/HW5Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys
import math

import matplotlib.pyplot as plt
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#Code for Part 1, solver
def LU(A):                        #this function returns L and U, where A=LU
  A = A.astype(float)
  L=np.identity(len(A))

  for i in range(0,len(A)):       #iterate through diagonal elements, starting at row/column 0
    for j in range(i+1, len(A)):  #iterate through rows below i
      if A[j,i]!=0:               #if the row does not have a leading 0, make it 0
        ratio=-A[j,i]/A[i,i]      #figure out what fraction of the ith row you need to add to the jth row so that the jth row will have a leading 0
        L[j,i]=-ratio             #keep track what you multiplied by in the L matrix
        A[j]=A[j]+A[i]*ratio      #the jth row becomes the jth row plus (ith row* ratio). The jth row now has a leading 0
 
  return L, A                     #at the end of this process, L is a lower triangular matrix, and A has been changed to an upper triangular matrix.

def solve(A, b):                  #this function returns x, if Ax=b. This is assuming that no pivoting is needed for A
  L, U=LU(A)                      #get L and U
  Ux=np.zeros(len(A))             #Ax=b -> LUx=b, solve L(Ux)=b for Ux first 
  for i in range(0,len(A)):       #iterate from 0 to len(A)
    Ux[i]=b[i]                    #set Ux at i to b at i
    for j in range(0, i):         #subtract the previous terms of Ux multipled by the L coefficients
      Ux[i]-=L[i,j]*Ux[j]
  
  x=np.zeros(len(A))
  for i in reversed(range(0, len(A))):  #Solve for x given Ux and U
    for j in range(i, len(A)):          #iterate through all the higher terms of x
      x[i]+=U[i,j]*x[j]                 #sum up the U coefficients * the higher terms of x
    x[i]=(Ux[i]-x[i])/U[i, i]           #the ith x is equal to (Ux[i] minus the sum) divided by the leading coefficient
  
  return  np.atleast_2d(x).T            #return a column vector with the solution




In [ ]:
def exact(x, y, t):                           #gives the exact solution to the function U(x,y,t)
  return math.exp(-t)*(x**2+y**2)+1  
  

In [ ]:
def solu(time, size):                               #returns a matrix with the exact solution at any time
  deltax=1/(size-1)
  deltay=1/(size-1)
  out=np.zeros((size,size))                
  for i in range(0,size):
    for j in range(0,size):
      out[i,j]=exact(i*deltax, j*deltay, time)
  return out

In [ ]:
def getInitial(size):                         #set initial conditions for U(x,y,0). This function returns a size by size matrix with the values of U at every position
  deltax=1/(size-1)                           #if size is 21, deltax is 1/20. grid positions range from 0,.05,.1,.15.....95,1
  deltay=1/(size-1)
  U=np.zeros((size,size))
  U = U.astype(float)
  for i in range(0,size):
    for j in range(0,size):
      U[i,j]=exact(i*deltax, j*deltay, 0)     #set U[i,j] to the exact solution

  U=U.reshape(size*size,1)                    #reshape it from a matrix to a vector, U00,U01,U02,U03..U10,U11,U12.... 

  return U

In [ ]:
def getA(r, g, size):                         #create the banded matrix for U(n+1) 
  A=np.identity(size)
  A=np.concatenate((A,np.tile(np.zeros((size,size)), size-1)), axis=1)
  
  D=np.identity(size)
  E=np.zeros((size, size))
  zeros=0
  
  for i in range(1, size-1):
    D[i, zeros]=r
    D[i, zeros+1]=g
    D[i, zeros+2]=r
    E[i, zeros+1]=r
    zeros+=1
  
  zeros=0
  for i in range(0, size-2):
    temp=np.concatenate((np.tile(np.zeros((size, size)),i),E,D,E,np.tile(np.zeros((size, size)),size-3-i)), axis=1)
    A=np.concatenate((A,temp),axis=0)
  
  temp=np.concatenate((np.tile(np.zeros((size, size)),size-1), np.identity(size)), axis=1)

  A=np.concatenate((A,temp), axis=0)
  return A

In [ ]:
def getB(r, g, size):                         #create the banded matrix for U(n)
  A=np.zeros((size,size))
  A=np.concatenate((A,np.tile(np.zeros((size,size)), size-1)), axis=1)
  
  D=np.zeros((size, size))
  E=np.zeros((size, size))
  zeros=0
  
  for i in range(1, size-1):
    D[i, zeros]=r
    D[i, zeros+1]=g
    D[i, zeros+2]=r
    E[i, zeros+1]=r
    zeros+=1
  
  zeros=0
  for i in range(0, size-2):
    temp=np.concatenate((np.tile(np.zeros((size, size)),i),E,D,E,np.tile(np.zeros((size, size)),size-3-i)), axis=1)
    A=np.concatenate((A,temp),axis=0)
  
  temp=np.concatenate((np.tile(np.zeros((size, size)),size-1), np.zeros((size, size))), axis=1)

  A=np.concatenate((A,temp), axis=0)
  return A

In [ ]:
def getC(n, size, deltat):                                          #create the vector of constants/boundry conditions
  deltax=1/(size-1)
  deltay=1/(size-1)
  C=np.zeros((size,size))                                           #make C initially a matrix, reshape to vector later

  for i in range(0, size):    
    for j in range(0, size):
      if  not (i ==0 or j==0 or i == size-1 or j == size -1):       #if i,i is not on a boundry (x=0,y=0,x=1,y=1)
        C[i,j]=-math.exp(-n*deltat)*(4+(i*deltax)**2+(j*deltay)**2) #C[i,j] is equal to the differential term -e^(-t)*(4+x^2+y^2)
      else:
        C[i,j]=exact(i*deltax, j*deltay, n*deltat)                  #C[i,j] is equal to the exact boundry condition if i,j is on a boundry

  return C.reshape(size*size,1)                                     #reshape C into a vector


In [ ]:
def getU(time):                               #returns U at the given time, using a 21x21 gird (x,y from 0 to 1 inclusive, deltax=deltay=1/20), deltat = .01
  size=21                                     #set some parameters
  deltax=1/(size-1)
  deltat=.01
  deltay=1/(size-1)


  r=-1/(2*deltax**2)                          #coefficients
  g=2/(deltax**2)+1/deltat
  h=1/deltat-2/(deltax**2)

  U=getInitial(size)                          #get the matricies
  A=getA(r,g, size)
  B=getB(-r,h,size)

  for n in range(1, 1+round(time/deltat)):        #iterate through time from 0 to t=.2(n from 0 to 20)
    C=getC(n, size, deltat)                     
    lhs=np.matmul(B, U)+C                       #the left hand side of the system is B * U at n plus C
    U=solve(A, lhs)                   #solve matrix equation Ax=lhs

  U=U.reshape(size,size)                        #resize U to a size by size matrix


  return U




In [ ]:
exactfinal=solu(.2, 21)       #get exact and numerical solutions at t=.2 to compare
numericalfinal=getU(.2)

exactmid=solu(.1,21)          #get exact and numerical solutions at t=.1 to compare
numericalmid=getU(.1)

print("Numerical solution at T=.2")        #compare error
print(np.around((exactfinal), 5))


print("Error of numerical solution at T=.2")        #compare error
print(np.around((numericalfinal-exactfinal),4))
print("Error of numerical solution at T=.1")
print(np.around((numericalmid-exactmid),4))
print("Numerical solution at T=.2")
plt.imshow(numericalfinal)        #plot numerical solution at t=.2
plt.colorbar()
plt.gca().invert_yaxis()
plt.clim(1, 2.75)
plt.show()
print("Exact solution at T=.2")
plt.imshow(exactfinal)      #plot exact solution at t=.2
plt.colorbar()
plt.clim(1, 2.75)
plt.gca().invert_yaxis()
plt.show()
print("Percent Error at T=.2")
plt.imshow(100*(numericalfinal-exactfinal)/exactfinal)      #plot percent error 
plt.colorbar()
plt.gca().invert_yaxis()
plt.show()
